This code consists of the first (initial data analysis) and second step (data structure improvement) of the methods described in our paper
First the  raw data is read from data set 1 (drug prescriptions) and data set 2 (drug catalogue).
Afterwards the data set 1 is checked in detail as follows:
- total drug prescriptions of the years 2016 to 2020
- determine no drug entries in the drug prescription data 
- determine amount of unstructured drug prescription data
- group unstructured drug prescription data by medication text and frequency is calculated
- check total number of different unstructured entries for the medication text
- evaluate distribution of free text drug prescriptions based on frequency
- check whether the first 1000 most frequent unstructured drug prescriptions are sufficient for manual evalution to reach the goal of 80% total medication orders having ATC code available 
- runs the algorithm on the unstructured data to determine ATC code in STEP1(regex medication product) , STEP2 (ingredient) and STEP 3 (NLP based on similariy with Levenshtein distance) - STEP 3 provides up to 3 different suggested ATC codes
- return results and create most frequent 1000 entries and results for algorithm 1, 2 and 3

In [ ]:
import pandas as pd
import numpy as np
import os
from fuzzywuzzy import fuzz, process
import re
from typing import List
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3, venn3_circles

# remove later
import warnings
warnings.filterwarnings('ignore')

cwd = os.getcwd()
# read data set 2 drug product catalogue. The encoding parameter is important.
df_catalogue = pd.read_csv('../../data_in/FINAL2_catalogue_20211116.csv', sep=";", encoding = "UTF-8").fillna(str())
# read data set 1, all drug prescription data
df_med_orders = pd.read_csv('../../data_in/FINAL_medication_orders.csv', skipinitialspace=True, low_memory=False, lineterminator='\n').fillna(str())
df_med_orders = df_med_orders[~df_med_orders['MEDICATION'].isnull()]
df_med_orders['MEDICATION'] = df_med_orders['MEDICATION'].astype(str)

ATC = 'ATC-DIM'
ABDA = 'ABDA-Bezeichnung'

df_med_orders.loc[df_med_orders["CAT_MATCH"] == "Fuzzy Match", 'CAT_MATCH'] = "No Match"

#limit drug prescriptions to the year 2016 - 2020
df_med_orders["START_DATE"] = pd.to_datetime(df_med_orders["START_DATE"])
df_med_orders = df_med_orders[~((df_med_orders["START_DATE"].dt.year < 2016) | (df_med_orders["START_DATE"].dt.year > 2020))]

df_catalogue['ATC-WHO'] = [i[1:] for i in df_catalogue['ATC-WHO']]

df_med_orders_complete = df_med_orders

In [ ]:
#check total number of entries in medication orders in scope (2016-2020)
print("total number of orders for all years from 2016 until 2020: ",len(df_med_orders))

#group total number of entries in medication orders by year and structure (structured/match oder unstructured/no match)
#df_med_orders.groupby("CAT_MATCH")["year"].value_counts()
df_med_orders.groupby(['CAT_MATCH']).count()

In [ ]:
# Filter: remove entries from df_med_orders that are no drug prescriptions based on simple rules
# remove all rows which start with a exclamation mark
no_med = df_med_orders[(df_med_orders['MEDICATION'].str.strip().str.startswith('BE ') 
          | df_med_orders['MEDICATION'].str.strip().str.startswith('BE:')
          | df_med_orders['MEDICATION'].str.strip().str.startswith('1 BE')
          | df_med_orders['MEDICATION'].str.strip().str.startswith('BB ')
          | df_med_orders['MEDICATION'].str.strip().str.startswith("!"))]


df_med_orders = df_med_orders[~(df_med_orders['MEDICATION'].str.strip().str.startswith('BE ') 
          | df_med_orders['MEDICATION'].str.strip().str.startswith('BE:')
          | df_med_orders['MEDICATION'].str.strip().str.startswith('1 BE')
          | df_med_orders['MEDICATION'].str.strip().str.startswith('BB ')
          | df_med_orders['MEDICATION'].str.strip().str.startswith("!"))]

#check total number of entries in medication orders in scope (2016-2020) with removed non medication entries based on above applied rules
print("total number of orders for all years from 2016 until 2020 without no medication entries determined by rule set: ",len(df_med_orders))
print(len(no_med))

In [ ]:
df_med_orders.groupby(['CAT_MATCH']).count()

In [ ]:
#here we check amount of structured versus unstructured data for 2016 to 2020 drug prescriptions
ana_med_orders = df_med_orders

#print(ana_med_orders.groupby("CAT_MATCH").size())

df_struct = ana_med_orders.loc[ana_med_orders['CAT_MATCH'] == "Match"]
print("total number of orders for all years from 2016 until 2020 STRUCTURED: ",len(df_struct))
df_unstruct = ana_med_orders.loc[ana_med_orders['CAT_MATCH'] == "No Match"]
print("total number of orders for all years from 2016 until 2020 UNSTRUCTURED: ",len(df_unstruct))

#group unstructured drug prescriptions and calculate cumsum in a separate column
#this is to prepare visualization of first 1000 rows of unstructured drug prescriptions and its frequency by medication text

df_unstruct['MEDICATION'] = df_unstruct['MEDICATION'].str.strip()
df_unstruct = df_unstruct.groupby(['MEDICATION', 'ATC_CODE', 'CAT_MATCH'], as_index=False).size().sort_values(by='size', ascending=False)
df_unstruct["cum_sum"] = df_unstruct['size'].cumsum()

Total = df_unstruct['size'].sum()
df_unstruct["percent"] = (df_unstruct['cum_sum'] / df_unstruct['size'].sum()) * 100
df_unstruct = df_unstruct.reset_index(drop=True)
df_unstruct

print("this is the percentage of frequency reached at row 1000: ", df_unstruct["percent"].iloc[999].round(decimals = 2))
print("this is the number of unstructured medication order amount at row 1000: ", df_unstruct["cum_sum"].iloc[999])


print("This is the percentage of total medication orders evaluated by taking the first 1000 most frequent unstructured medication orders for manual evaluation  and the already structured medication orders: ",(((df_unstruct['cum_sum'].iloc[999]+len(df_struct)+len(no_med)) / len(df_med_orders_complete)) * 100).round(decimals = 2) )
#this plot is Figure3 in the paper
myplot = sns.lineplot(x = df_unstruct.index, y = "percent", data=df_unstruct)
myplot.vlines(x=1000, ymin=0, ymax=100, colors='r', linewidth=0.5)
myplot.hlines(y=65.5666041423044, xmin=0, xmax=100000, colors='r', linewidth=0.5)
65.5666041423044
myplot.set_xlabel('frequency of unstructured medication order grouped by MEDICATION text')
figure = myplot.get_figure()   
figure.savefig('Top1000.png', dpi=400, bbox_inches = "tight",figsize=(10, 2))

In [ ]:
df_med_orders['MEDICATION'] = df_med_orders['MEDICATION'].str.strip()
df_med_orders = df_med_orders.groupby(['MEDICATION', 'ATC_CODE', 'CAT_MATCH'], as_index=False).size().sort_values(by='size', ascending=False)


# remove all rows of catalogue which do not have a given atc code
df_catalogue = df_catalogue[df_catalogue[ATC] != str()]

print("total number of orders grouped by MEDICATION text field: ",len(df_med_orders))

In [ ]:
abda = df_catalogue[ABDA]

df_catalogue['ingredient'] = [str(ing).lower() if str(ing2) == str() else str(ing2).lower() 
                              for ing, ing2 in zip(df_catalogue['Wirkstoff'], df_catalogue['enstpr. Wirkstoff'])]

# add boolean if ingredient is specified
df_med_orders['PIPE'] = [int(True) if "|" in med else int(False) for med in df_med_orders['MEDICATION']]

In [ ]:
def print_array(arr : List) -> str:
    if isinstance(arr, str):
        return arr
    
    s = str()
    for i in arr:
        s += str(i) + " " 
    
    return s.rstrip()

In [ ]:
# extract ingredient -> format drugs etc. | (ingredient)

def extract_ingredient(m : str) -> str:
    med = m.split("|")
    
    # was there a pipe in the string?
    if len(med) == 1:
        return str()
    
    # return ingredient in lower case and without brackets
    return re.sub('\s+',' ', med[-1]).lower().strip()[1:-1]
    
# if result is an empty string ingredient could still be in the medication name like ibuprofen
df_med_orders['INGREDIENT'] = [extract_ingredient(x) for x in df_med_orders['MEDICATION'].tolist()]

In [ ]:
def arr_to_len_three(arr):
    arr = arr + [str()] * 3
    return arr[:3]

In [ ]:
def algorithm(med : str, ing : str) -> List[str]:
    
    if ing != str():
        cat = df_catalogue[df_catalogue['ingredient'] == ing]
        
        # STEP 1 
        step1 = np.unique(cat[ATC])
        step1 = step1[0] if len(step1) == 1 else str()
           
    else: 
        cat = list()
        step1 = str()
    
    
    # remove the ingredient out of medication name -> not important for now because we extracted it
    med = med.split('|')[0]
    
    med_spl = med.split()
    # if there are more than 3 words, than just take the first 3, because these informations are more relevant 
    # and give better results
    if len(med_spl) > 3:
        med = print_array(med_spl[:3])

    
    # if there are no results 
    if not len(cat):
        res_med = process.extract(med, abda, limit=3, scorer=fuzz.token_set_ratio)
         
    else:
        res_med = process.extract(med, cat[ABDA].to_list(), limit=3, scorer=fuzz.token_set_ratio)
        
    
    # we only need catalogue as df -> medication should only exist once in catalogue for the exact same string
    df = pd.DataFrame()

    for i in range(len(res_med)):
        df = df.append(df_catalogue[df_catalogue[ABDA] == res_med[i][0]].iloc[0][['ABDA-Bezeichnung', 'ATC-DIM', 'ATC-WHO', 'DFO Bezeichnung', 'SAP-Kurztext']])
    
    # append the calculated values of the levenshtein distanc
    df['value_lev'] = [i[1] for i in res_med]
        
    df = df.drop_duplicates(subset=[ATC])

    # row = df[df[ATC] == codes[0]].iloc[0]
    
    # STEP 2
    step2 = list(np.unique(df_catalogue[df_catalogue[ABDA].str.contains(med, case=False, regex=False)][ATC]))
    step2 = step2[0] if len(step2) == 1 else str()

    
    
    # return -> all ATC-DIM, ATC-WHO, Levenshtein Distances,      
    return (step1,
            step2,
            arr_to_len_three(list(df[ATC])), 
            arr_to_len_three(list(df['ATC-WHO'])), 
            arr_to_len_three(list(df['value_lev'].astype(int))))

In [ ]:
def compare_rows(atc1, atc2, atc3=False):
    if atc3 == False:
        atc3 = atc1
    
    if pd.isnull(atc1) or pd.isnull(atc2) or pd.isnull(atc3):
        return 0
    elif atc1 == atc2 and atc1 == atc3:
        return 1
    else: #  np.isnan(atc1) or np.isnan(atc2) or not (atc1 == atc2):
        return 2

In [ ]:
# check all given atc codes for accuracy based Matches entries (structured data)

def main(df):
    # if ATC_CODE1 == ATC_CODE2 we got the right one
    print(f'Length df -> : {len(df)}')

    match = []
    
    meds = list(df['MEDICATION'])

    df['INGREDIENT'] = [extract_ingredient(x) for x in meds]
    
    df = df.reset_index()
    
    for index, med in enumerate(tqdm(zip(df['MEDICATION'], df['INGREDIENT']))):
        
        arr = algorithm(med[0], med[1])
        # print(f'arr --> {arr}')
                
        df.at[index, ['STEP1', 'STEP2']] = arr[:2]
        df.at[index, ['STEP31', 'STEP32', 'STEP33']] = arr[2]
        df.at[index, ['LEV1', 'LEV2', 'LEV3']] = arr[4]
    
    df['TRUE12'] = [compare_rows(atc1, atc2) for atc1, atc2 in zip(df['STEP1'], df['STEP2'])]                   
    df['TRUE123'] = [compare_rows(atc1, atc2, atc3) for atc1, atc2, atc3 in zip(df['STEP1'], df['STEP2'], df['STEP31'])]
    df['TRUE13'] = [compare_rows(atc1, atc2) for atc1, atc2 in zip(df['STEP1'], df['STEP31'])]
    df['TRUE23'] = [compare_rows(atc1, atc2) for atc1, atc2 in zip(df['STEP2'], df['STEP31'])]
    df['MATCH'] = [compare_rows(atc1, atc2) for atc1, atc2 in zip(df['ATC_CODE'], df['STEP31'])]

    # adding first atc level as extra column
    #df['ATC_LVL1_JOS'] = [atc[:1] for atc in df['ATC_JOS1']]
    #df['ATC_LVL1_MAR'] = [atc[:1] for atc in df['ATC_CODE']]
    df['LEV1'] = df['LEV1'].astype(int)
    
    return df

In [ ]:
df = df_med_orders
res = main(df)
del res['index']

# adding first atc level as extra column
res['STEP31_LVL1'] = [atc[:1] for atc in res['STEP31']]
res['ATC_CODE_LVL1'] = [atc[:1] for atc in res['ATC_CODE']]
#export complete results here
res.to_csv('../../data_in/res.csv', index=False)

In [ ]:
res_un = res.loc[res['CAT_MATCH'] == "No Match"]

In [ ]:
res1000 = res_un.head(n=1000)

In [ ]:
# export complete result list of the most frequent 1000 free text drug prescriptions here
res1000.to_csv('../../data_in/res1000.csv', index=False)

In [ ]:
print("Number of of different txt entries algorithm 1 identified an ATC code: ", len(res_un.loc[res_un['STEP1']!='']))
print("Total number of drug prescriptions algorithm 1 identified an ATC code: ", res_un["size"].loc[res_un['STEP1']!=''].sum())
print("Number of of different txt entries algorithm 2 identified an ATC code: ", len(res_un.loc[res_un['STEP2']!='']))
print("Total number of drug prescriptions algorithm 2 identified an ATC code: ", res_un["size"].loc[res_un['STEP2']!=''].sum())
print("Number of of different txt entries algorithm 3 identified an ATC code: ", len(res_un.loc[res_un['STEP31']!='']))
print("Total number of drug prescriptions algorithm 3 identified an ATC code: ", res_un["size"].loc[res_un['STEP31']!=''].sum())
print("****************************************************************************************************************")
print("Number of of different txt entries algorithm 1 + 2 results match: ", len(res_un.loc[res_un['TRUE12']==1].loc[res_un['STEP1']!='']))
print("Total number of drug prescriptions algorithm 1 + 2 results match: ", res_un["size"].loc[res_un['TRUE12']==1].loc[res_un['STEP1']!=''].sum())
print("Number of of different txt entries algorithm 1 + 3 results match: ", len(res_un.loc[res_un['TRUE13']==1].loc[res_un['STEP1']!='']))
print("Total number of drug prescriptions algorithm 1 + 3 results match: ", res_un["size"].loc[res_un['TRUE13']==1].loc[res_un['STEP1']!=''].sum())
print("Number of of different txt entries algorithm 2 + 3 results match: ", len(res_un.loc[res_un['TRUE23']==1].loc[res_un['STEP2']!='']))
print("Total number of drug prescriptions algorithm 2 + 3 results match: ", res_un["size"].loc[res_un['TRUE23']==1].loc[res_un['STEP2']!=''].sum())
print("Number of of different txt entries algorithm 1 + 2 + 3 results match: ", len(res_un.loc[res_un['TRUE123']==1].loc[res_un['STEP2']!='']))
print("Total number of drug prescriptions algorithm 1 + 2 + 3 results match: ", res_un["size"].loc[res_un['TRUE123']==1].loc[res_un['STEP2']!=''].sum())
print("****************************************************************************************************************")
print("TOP 1000 - Number of of different txt entries algorithm 1 + 2 results match: ", len(res1000.loc[res1000['TRUE12']==1].loc[res1000['STEP1']!='']))
print("TOP 1000 - Total number of drug prescriptions algorithm 1 + 2 results match: ", res1000["size"].loc[res1000['TRUE12']==1].loc[res1000['STEP1']!=''].sum())
print("TOP 1000 - Number of of different txt entries algorithm 1 + 3 results match: ", len(res1000.loc[res1000['TRUE13']==1].loc[res1000['STEP1']!='']))
print("TOP 1000 - Total number of drug prescriptions algorithm 1 + 3 results match: ", res1000["size"].loc[res1000['TRUE13']==1].loc[res1000['STEP1']!=''].sum())
print("TOP 1000 - Number of of different txt entries algorithm 2 + 3 results match: ", len(res1000.loc[res1000['TRUE23']==1].loc[res1000['STEP2']!='']))
print("TOP 1000 - Total number of drug prescriptions algorithm 2 + 3 results match: ", res1000["size"].loc[res1000['TRUE23']==1].loc[res1000['STEP2']!=''].sum())
print("TOP 1000 - Number of of different txt entries algorithm 1 + 2 + 3 results match: ", len(res1000.loc[res1000['TRUE123']==1].loc[res1000['STEP2']!='']))
print("TOP 1000 - Total number of drug prescriptions algorithm 1 + 2 + 3 results match: ", res1000["size"].loc[res1000['TRUE123']==1].loc[res1000['STEP2']!=''].sum())
print("****************************************************************************************************************")
print("TOP 1000 - Number of of different txt entries algorithm 1 identified an ATC code: ", len(res1000.loc[res1000['STEP1']!='']))
print("TOP 1000 - Total number of drug prescriptions algorithm 1 identified an ATC code: ", res1000["size"].loc[res1000['STEP1']!=''].sum())
print("TOP 1000 - Number of of different txt entries algorithm 2 identified an ATC code: ", len(res1000.loc[res1000['STEP2']!='']))
print("TOP 1000 - Total number of drug prescriptions algorithm 2 identified an ATC code: ", res1000["size"].loc[res1000['STEP2']!=''].sum())
print("TOP 1000 - Number of of different txt entries algorithm 3 identified an ATC code: ", len(res1000.loc[res1000['STEP31']!='']))
print("TOP 1000 - Total number of drug prescriptions algorithm 3 identified an ATC code: ", res1000["size"].loc[res1000['STEP31']!=''].sum())